In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import imblearn
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('M1_final.csv')
df_pre = df.copy()

df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,DEST,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,...,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,sch_dep,sch_arr,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,...,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,...,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,...,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,...,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,...,32,58,W,24,35,29.91,Fair / Windy,9,17,13


In [3]:
df.shape

(28820, 23)

In [4]:
df.describe()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,DEP_TIME_M,CRS_ARR_M,Temperature,Humidity,Wind Speed,Wind Gust,Pressure,sch_dep,sch_arr,TAXI_OUT
count,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000,28820.000000
mean,7.894240,16.021096,4.008952,6.374983,225.288203,1267.746079,831.003851,828.934698,910.874289,41.489833,57.732616,12.367627,5.535323,30.092433,31.091256,28.432130,20.858570
std,4.991723,8.750179,1.985230,38.735144,119.482417,889.343246,299.398525,305.864103,345.411743,8.043533,23.468676,6.259298,11.886457,0.296160,9.510359,8.263043,6.851915
min,1.000000,1.000000,1.000000,-22.000000,57.000000,94.000000,301.000000,1.000000,1.000000,17.000000,0.000000,0.000000,0.000000,29.200000,0.000000,0.000000,5.000000
25%,1.000000,8.000000,2.000000,-6.000000,124.000000,483.000000,545.000000,542.000000,667.000000,36.000000,46.000000,8.000000,0.000000,29.880000,26.000000,21.000000,16.000000
50%,11.000000,16.000000,4.000000,-3.000000,188.000000,1029.000000,856.000000,854.000000,918.000000,42.000000,59.000000,12.000000,0.000000,30.110000,30.000000,30.000000,19.000000
75%,12.000000,24.000000,6.000000,2.000000,365.000000,2248.000000,1095.000000,1097.000000,1193.000000,47.000000,74.000000,16.000000,0.000000,30.320000,37.000000,35.000000,25.000000
max,12.000000,31.000000,7.000000,1276.000000,697.000000,4983.000000,1439.000000,1440.000000,1439.000000,68.000000,97.000000,36.000000,49.000000,30.750000,55.000000,46.000000,41.000000


In [5]:
# all columns of df
df.columns

Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'TAIL_NUM',
       'DEST', 'DEP_DELAY', 'CRS_ELAPSED_TIME', 'DISTANCE', 'CRS_DEP_M',
       'DEP_TIME_M', 'CRS_ARR_M', 'Temperature', 'Dew Point', 'Humidity',
       'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Condition', 'sch_dep',
       'sch_arr', 'TAXI_OUT'],
      dtype='object')

In [6]:
# Label encoding for OP_UNIQUE_CARRIER\n",
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['OP_UNIQUE_CARRIER'] = le.fit_transform(df['OP_UNIQUE_CARRIER'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'9E': 0, 'AA': 1, 'AS': 2, 'B6': 3, 'DL': 4, 'HA': 5, 'MQ': 6, 'OO': 7, 'YX': 8}


Mostly,heavy - 3
Partly, Light - 2
columns - cloudy, windy, fair, rain,fog, drizzle, snow, wintry mix , freezing rain,

In [7]:
#add columns of cloudy, windy, fair, rain,fog, drizzle, snow, wintry mix , freezing rain,
df['Cloudy'] = df['Condition'].apply(lambda x: 3 if re.search(r'Mostly Cloudy', x) else 1 if re.search(r'Partly Cloudy', x) else 2 if re.search(r'Cloudy', x) else 0)
df['Windy'] = df['Condition'].apply(lambda x: 1 if re.search(r'Windy', x) else 0)
df['Fair'] = df['Condition'].apply(lambda x: 1 if re.search(r'Fair', x) else 0)
df['Rain'] = df['Condition'].apply(lambda x: 3 if re.search(r'Heavy Rain', x) else 2 if re.search(r'Rain', x) else 1 if re.search(r'Light Rain', x) else 0)
df['Fog'] = df['Condition'].apply(lambda x: 1 if re.search(r'Fog', x) else 0)
df['Drizzle'] = df['Condition'].apply(lambda x: 2 if re.search(r'Drizzle', x) else 1 if re.search(r'Light Drizzle', x) else 0)
df['Snow'] = df['Condition'].apply(lambda x: 3 if re.search(r'Heavy Snow', x) else 2 if re.search(r'Snow', x) else 1 if re.search(r'Light Snow', x) else 0)
df['Wintry Mix'] = df['Condition'].apply(lambda x: 1 if re.search(r'Wintry Mix', x) else 0)
df['Freezing Rain'] = df['Condition'].apply(lambda x: 2 if re.search(r'Freezing Rain', x) else 1 if re.search(r'Light Freezing Rain', x) else 0)

In [8]:
#Cyclic encoding for Day of week
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)
    return data


df = encode(df, 'MONTH', 12)
df = encode(df, 'DAY_OF_MONTH', 31)
df = encode(df, 'DAY_OF_WEEK', 7)

df_pre = df.copy()
# make dep delay as binary
df_pre['DEP_DELAY'] = df_pre['DEP_DELAY'].apply(lambda x: 1 if x > 15 else 0)

# df_pre  = df.copy()
lb = LabelEncoder()
lb_dest = LabelEncoder()
df_pre['DEST'] = lb_dest.fit_transform(df_pre['DEST'])
lb_name_mapping_dest = dict(zip(lb_dest.classes_, lb_dest.transform(lb_dest.classes_)))
print(lb_name_mapping_dest)
lb_dew = LabelEncoder()
df_pre['Dew Point'] = lb_dew.fit_transform(df_pre['Dew Point'])
lb_name_mapping_dew = dict(zip(lb_dew.classes_, lb_dew.transform(lb_dew.classes_)))
print(lb_name_mapping_dew)
lb_wind = LabelEncoder()
df_pre['Wind'] = lb_wind.fit_transform(df_pre['Wind'])
lb_name_mapping_wind = dict(zip(lb_wind.classes_, lb_wind.transform(lb_wind.classes_)))
print(lb_name_mapping_wind)

df_pre.drop(['TAIL_NUM'], axis=1, inplace=True)
x = df_pre.drop(['DEP_DELAY'], axis=1)
y = df_pre['DEP_DELAY']

print(df_pre.info())
print(df_pre.shape)

df_pre.drop(['Condition'], axis=1, inplace=True)
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(df_pre)

df_pre.head()
# save df_pre as a new csv file
df_pre.to_csv('df_preprocessed.csv', index=False)

{'ABQ': 0, 'ATL': 1, 'AUS': 2, 'BNA': 3, 'BOS': 4, 'BQN': 5, 'BTV': 6, 'BUF': 7, 'BUR': 8, 'BWI': 9, 'CHS': 10, 'CLE': 11, 'CLT': 12, 'CMH': 13, 'CVG': 14, 'DCA': 15, 'DEN': 16, 'DFW': 17, 'DTW': 18, 'EGE': 19, 'FLL': 20, 'HNL': 21, 'IAD': 22, 'IAH': 23, 'IND': 24, 'JAC': 25, 'JAX': 26, 'LAS': 27, 'LAX': 28, 'LGB': 29, 'MCO': 30, 'MIA': 31, 'MSP': 32, 'MSY': 33, 'OAK': 34, 'ONT': 35, 'ORD': 36, 'ORF': 37, 'ORH': 38, 'PBI': 39, 'PDX': 40, 'PHL': 41, 'PHX': 42, 'PIT': 43, 'PSE': 44, 'PSP': 45, 'PWM': 46, 'RDU': 47, 'RIC': 48, 'RNO': 49, 'ROC': 50, 'RSW': 51, 'SAN': 52, 'SAT': 53, 'SAV': 54, 'SEA': 55, 'SFO': 56, 'SJC': 57, 'SJU': 58, 'SLC': 59, 'SMF': 60, 'SRQ': 61, 'STT': 62, 'SYR': 63, 'TPA': 64}
{'-1': 0, '-2': 1, '-3': 2, '0\xa0': 3, '10': 4, '11': 5, '12': 6, '13': 7, '14': 8, '15': 9, '16': 10, '17': 11, '18': 12, '19': 13, '1\xa0': 14, '20': 15, '21': 16, '22': 17, '23': 18, '24': 19, '25': 20, '26': 21, '27': 22, '28': 23, '29': 24, '2\xa0': 25, '30': 26, '31': 27, '32': 28, '33'